#### CSML1020 Course Project - New Plant Diseases Dectection
#### Authors (Group 3): Paul Doucet, Jerry Khidaroo

#### Initilization

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# import seaborn as sns
import os

In [4]:
import tensorflow as tf


In [6]:
# Use this to disable GPU
tf.config.set_visible_devices([], 'GPU')

In [7]:
from keras.applications.vgg16 import VGG16
import keras
from sklearn.model_selection import train_test_split
from keras import backend
from keras.layers import Dense
from keras.layers import Flatten
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D
from keras.optimizers import SGD
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.layers import Dropout
from keras.layers.normalization import BatchNormalization

Using TensorFlow backend.


#### Data Importing

In [12]:
# Image Data Folders
# dataDirTrain = '../NewPlantDiseasesDatasetSample/train'
# dataDirValidate = '../NewPlantDiseasesDatasetSample/valid'
# dataDirTest = '../NewPlantDiseasesDatasetSample/test'

dataDirTrain = '../NewPlantDiseasesDataset/train'
dataDirValidate = '../NewPlantDiseasesDataset/valid'
dataDirTest = '../NewPlantDiseasesDataset/test'

#### Load Train Image Data with Best Augmentation Filters

In [13]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255, zoom_range=0.2, fill_mode='nearest')
valid_datagen = ImageDataGenerator(rescale=1./255)

# batch_size = 128
batch_size = 48
training_set = train_datagen.flow_from_directory(dataDirTrain, target_size=(224, 224), batch_size=batch_size, class_mode='categorical')
valid_set = valid_datagen.flow_from_directory(dataDirValidate, target_size=(224, 224), batch_size=batch_size, class_mode='categorical')

train_num = training_set.samples
valid_num = valid_set.samples

Found 70295 images belonging to 38 classes.
Found 17572 images belonging to 38 classes.


#### Perform Hyper-Parameter Tuning on Selected Model

In [14]:
from keras.optimizers import Adam
from keras.applications.vgg16 import VGG16
import keras

def create_model(activation='softmax', learning_rate=0.01):
    base_model=VGG16(include_top=False,input_shape=(224,224,3))
    # base_model=VGG16(include_top=False,input_shape=(112,112,3))
    base_model.trainable=False

    classifier=keras.models.Sequential()
    classifier.add(base_model)
    classifier.add(Flatten())
    #classifier.add(Dense(38,activation='softmax'))
    classifier.add(Dense(38,activation=activation))

    opt = Adam(lr=learning_rate)
    
    #classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    classifier.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

    return classifier

In [15]:
import keras
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import RandomizedSearchCV, KFold, cross_val_score
from keras import regularizers

# Create a KerasClassifier
model = KerasRegressor(build_fn = create_model, verbose = 0)

# Define the parameters to try out
params = {'activation': ["softmax", "tanh"],
          'batch_size': [32, 48, 64], 
          'epochs': [3, 5, 10],
          'learning_rate': [0.01, 0.001, 0.0001]}

# Create a randomize search cv object passing in the parameters to try
random_search = RandomizedSearchCV(model,
                                   param_distributions = params,
                                   cv = KFold(3))

X_train, y_train = next(training_set)

random_search_results = random_search.fit(X_train, y_train)

# Print best score and parameters
print("Best Score: ", random_search_results.best_score_, "and Best Params: ", random_search_results.best_params_)

Best Score:  -3.752800544102987 and Best Params:  {'learning_rate': 0.001, 'epochs': 10, 'batch_size': 32, 'activation': 'tanh'}


In [16]:
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score
from keras.optimizers import Adam
from keras import regularizers

# Create a KerasClassifier
model = KerasRegressor(build_fn = create_model, verbose = 0)

# Define the parameters to try out
params = {'activation': ["softmax", "tanh"],
          'batch_size': [32, 48, 64], 
          'epochs': [3, 5, 10],
          'learning_rate': [0.01, 0.001, 0.0001]}

# Create a grid search cv object passing in the parameters to try
random_search = GridSearchCV(model, params, cv = KFold(3))

X_train, y_train = next(training_set)

grid_search_results = random_search.fit(X_train, y_train)

# Print best score and parameters
print("Best Score: ", grid_search_results.best_score_, "and Best Params: ", grid_search_results.best_params_)

Best Score:  -3.7668916384379068 and Best Params:  {'activation': 'tanh', 'batch_size': 48, 'epochs': 3, 'learning_rate': 0.001}


#### Inference